<center>

# **Join Files**

</center>

This algorith join all the indicators downloaded from
https://ghdx.healthdata.org/record/global-burden-disease-study-2019-gbd-2019-covariates-1980-2019


In [157]:
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt
from collections import Counter
import requests
import io
import os

In [158]:
#Mount your Google Drive to Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Health Indicators filtered from WHO database

In [159]:
output_path = '/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/'
file_name   = '2.Values_Health_Coverage_Indicators_WHO.xlsx'
file_path = os.path.join(output_path, file_name)

health_coverage_data = pd.read_excel(file_path)


health_coverage_data_filter = health_coverage_data[['IndicatorCode','SEX','Year','Country','Value']]
health_coverage_data_filter.head(5)

,IndicatorCode,SEX,Year,Country,Value
0,UHC_INDEX_REPORTED,NaN,2000,ARG,68.04714
1,UHC_INDEX_REPORTED,NaN,2005,ARG,73.82919
2,UHC_INDEX_REPORTED,NaN,2010,ARG,76.45497
3,UHC_INDEX_REPORTED,NaN,2015,ARG,79.66449
4,UHC_INDEX_REPORTED,NaN,2017,ARG,78.63055


## Step 1: Join all files in folder GBD_1980_2019_Data

In [160]:
folder_path = '/content/drive/MyDrive/2023_ODS_OPS/Data/GBD_1980_2019_Data/'

In [161]:
# Get a list of all Excel files in the folder
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]


In [105]:
#estos archivos #0,1,2,4,11 tienen information por grupo de edad

#file= excel_files[11]
#file_path = os.path.join(folder_path, file)
#df = pd.read_excel(file_path)
#print(df['IndicatorName'].unique())
#df.head(5)

In [162]:
reduce_excel_files = [excel_files[i] for i in [3,5,6,7,8,9,10]]

In [163]:
# Initialize an empty list to store DataFrames
dfs = []

# Read each Excel file and store it in the list
for file in reduce_excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)
    dfs.append(df)


In [164]:
# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.head(5)

,Unnamed: 0,IndicatorCode,Year,Country,Value,IndicatorName,sex_id,sex_label,age_group_id,age_group_name
0,0,pop_dens_150_300_psqkm_pct,1980,ARG,0.050821,Proportion of the country with population dens...,3,Both,22,All Ages
1,1,pop_dens_150_300_psqkm_pct,1981,ARG,0.050897,Proportion of the country with population dens...,3,Both,22,All Ages
2,2,pop_dens_150_300_psqkm_pct,1982,ARG,0.051060,Proportion of the country with population dens...,3,Both,22,All Ages
3,3,pop_dens_150_300_psqkm_pct,1983,ARG,0.051229,Proportion of the country with population dens...,3,Both,22,All Ages
4,4,pop_dens_150_300_psqkm_pct,1984,ARG,0.051365,Proportion of the country with population dens...,3,Both,22,All Ages


### Save indicator names

In [165]:
filter_df = combined_df[['IndicatorName','IndicatorCode']]
# Drop duplicated rows based on the selected columns
filter_df = filter_df.drop_duplicates()
filter_df.head(5)

,IndicatorName,IndicatorCode
0,Proportion of the country with population dens...,pop_dens_150_300_psqkm_pct
1400,Healthcare access and quality (HAQ) index,haqi
2800,Proportion of the country with population dens...,pop_dens_300_500_psqkm_pct
4200,Proportion of the country with population dens...,pop_dens_500_1000_psqkm_pct
5600,Proportion of the country with population dens...,pop_dens_over_1000_psqkm_pct


In [26]:
# Path to the output file where you want to save the combined data
output_file = '/content/drive/MyDrive/2023_ODS_OPS/list_extra_indicators.xlsx'

# Save the combined DataFrame to a new Excel file
filter_df.to_excel(output_file, index=False)


## Filter indicator related with health coverage and join with WHO Base

In [166]:
new_indicators=combined_df[['IndicatorCode',	'sex_label', 'Year',	'Country',	'Value']]
new_indicators.head(5)

# Change the name of the column from 'IndicatorName' to 'SEX'
new_indicators.rename(columns={'sex_label': 'SEX'}, inplace=True)

# Update the values 'female', 'male', and 'Both' to 'FMLE', 'MLE', and 'BOTH' respectively
new_indicators['SEX'] = new_indicators['SEX'].replace({
    'Female': 'FMLE',
    'Male': 'MLE',
    'Both': ''
})

# Filtering the data
new_indicators_filter_year = new_indicators[(new_indicators['Year'] >= 2000) & (new_indicators['Year'] <= 2019)]
new_indicators_filter_year.head(5)


<ipython-input-166-0536a8730efc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_indicators.rename(columns={'sex_label': 'SEX'}, inplace=True)
<ipython-input-166-0536a8730efc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_indicators['SEX'] = new_indicators['SEX'].replace({


,IndicatorCode,SEX,Year,Country,Value
20,pop_dens_150_300_psqkm_pct,,2000,ARG,0.043994
21,pop_dens_150_300_psqkm_pct,,2001,ARG,0.043995
22,pop_dens_150_300_psqkm_pct,,2002,ARG,0.044039
23,pop_dens_150_300_psqkm_pct,,2003,ARG,0.044060
24,pop_dens_150_300_psqkm_pct,,2004,ARG,0.044078


In [102]:
new_indicators_filter_year.to_excel('/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/5.Indicators_from_ghdx_healthdata.xlsx')

In [167]:
list_indicators_to_join_health_data = ['pop_dens_300_500_psqkm_pct', 'pop_dens_500_1000_psqkm_pct', 'pop_dens_over_1000_psqkm_pct', 'pop_dens_150_300_psqkm_pct', 'pop_dens_under_150_psqkm_pct', 'haqi']

indicators_health = new_indicators[new_indicators['IndicatorCode'].isin(list_indicators_to_join_health_data)]
indicators_health.head(5)

indicators_health['Year'].unique()

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [168]:
final_healht_data = pd.concat([health_coverage_data_filter,indicators_health])
final_healht_data['SEX'].unique()


array([nan, 'RUR', 'TOTL', 'URB', 'FMLE', 'MLE', ''], dtype=object)

In [169]:
# Filtering the data
final_healht_data_year_filter = final_healht_data[(final_healht_data['Year'] >= 2000) & (final_healht_data['Year'] <= 2019)]


In [152]:
final_healht_data_year_filter.to_excel('/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/2.Values_Health_Coverage_Indicators_WHO_gdhx_healthdata.xlsx')

In [170]:
final_healht_data_year_filter.to_csv('/content/drive/MyDrive/2023_ODS_OPS/OutputFiles/2.Values_Health_Coverage_Indicators_WHO_gdhx_healthdata.csv')